# Whisper Transcription Tool v2.0

## Version refactorisée et simplifiée

### Fonctionnalités :
- 🎥 Transcription de vidéos YouTube via URL
- 📁 Traitement par lot de dossiers de fichiers
- 🎵 Support natif des fichiers audio (MP3, WAV, AAC, FLAC, OGG, M4A)
- 🎬 Support des fichiers vidéo (MP4, AVI, MKV, WebM, MOV, FLV)
- 📝 Export en format texte (.txt) et sous-titres (.srt)
- 🚀 Détection automatique du GPU
- 🔧 Interface utilisateur simplifiée

### Améliorations v2.0 :
- ✅ Gestion GPU simplifiée
- ✅ Interface utilisateur interactive
- ✅ Détection automatique du type de fichier
- ✅ Code nettoyé et optimisé
- ✅ Workflow unifié pour audio et vidéo

## 1. Installation des dépendances

In [ ]:
# Installation des dépendances
!pip install -q ffmpeg yt_dlp huggingface_hub
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch torchaudio torchvision

## 2. Import des librairies

In [ ]:
import os
import subprocess
import glob
import zipfile
import time
import shutil
from pathlib import Path

import torch
import whisper
from yt_dlp import YoutubeDL
from huggingface_hub import HfApi

print("📦 Toutes les librairies importées avec succès !")

## 3. Fonctions utilitaires

In [ ]:
def format_timestamp(seconds):
    """Convertit les secondes en format timestamp SRT (HH:MM:SS,mmm)"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    milliseconds = int((seconds % 1) * 1000)
    return f"{hours:02}:{minutes:02}:{secs:02},{milliseconds:03}"

def detect_file_type(filepath):
    """Détecte automatiquement si le fichier est audio ou vidéo"""
    audio_extensions = ('.mp3', '.wav', '.aac', '.flac', '.ogg', '.m4a')
    video_extensions = ('.mp4', '.avi', '.mkv', '.webm', '.mov', '.flv')
    
    filepath_lower = filepath.lower()
    if filepath_lower.endswith(audio_extensions):
        return "audio"
    elif filepath_lower.endswith(video_extensions):
        return "video"
    else:
        return "unknown"

def check_ffmpeg():
    """Vérifie si ffmpeg est installé"""
    try:
        subprocess.run(["ffmpeg", "-version"], check=True, capture_output=True, text=True)
        return True
    except (FileNotFoundError, subprocess.CalledProcessError):
        return False

def setup_device():
    """Configuration automatique du device (GPU si disponible, sinon CPU)"""
    if torch.cuda.is_available():
        device = "cuda"
        gpu_name = torch.cuda.get_device_name(0)
        print(f"🚀 GPU détecté : {gpu_name}")
    else:
        device = "cpu"
        print("💻 Utilisation du CPU")
    return device

print("✅ Fonctions utilitaires définies")

## 4. Fonctions de traitement

In [ ]:
def download_video(url, output_dir):
    """Télécharge une vidéo depuis une URL"""
    print(f"📥 Téléchargement de la vidéo : {url}")
    
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
        'noplaylist': True,
        'quiet': True,
    }
    
    try:
        with YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            
        # Chercher le fichier téléchargé
        video_files = glob.glob(os.path.join(output_dir, "*"))
        video_files = [f for f in video_files if detect_file_type(f) in ["video", "audio"]]
        
        if video_files:
            video_path = max(video_files, key=os.path.getctime)  # Le plus récent
            print(f"✅ Téléchargement terminé : {os.path.basename(video_path)}")
            return video_path
        else:
            raise FileNotFoundError("Aucun fichier vidéo/audio trouvé après téléchargement")
            
    except Exception as e:
        print(f"❌ Erreur lors du téléchargement : {e}")
        return None

def convert_video_to_audio(video_path, audio_path):
    """Convertit une vidéo en fichier audio MP3"""
    print(f"🔄 Conversion : {os.path.basename(video_path)} → {os.path.basename(audio_path)}")
    
    if not check_ffmpeg():
        raise RuntimeError("ffmpeg n'est pas installé")
    
    try:
        command = [
            "ffmpeg", "-i", video_path,
            "-vn", "-acodec", "libmp3lame", "-q:a", "0",
            "-y", audio_path
        ]
        
        subprocess.run(command, check=True, capture_output=True, text=True)
        print("✅ Conversion terminée")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Erreur lors de la conversion : {e.stderr}")
        return False

def transcribe_audio(model, audio_path, device="auto"):
    """Transcrit un fichier audio avec Whisper"""
    print(f"🎤 Transcription : {os.path.basename(audio_path)}")
    
    try:
        result = model.transcribe(audio_path, language="fr", verbose=False)
        print("✅ Transcription terminée")
        return result
    except Exception as e:
        print(f"❌ Erreur lors de la transcription : {e}")
        return None

def save_transcription(result, base_path):
    """Sauvegarde la transcription en formats .txt et .srt"""
    if not result:
        return None, None
    
    text_file = base_path + ".txt"
    srt_file = base_path + ".srt"
    
    print(f"💾 Sauvegarde : {os.path.basename(text_file)} et {os.path.basename(srt_file)}")
    
    try:
        # Sauvegarde du texte
        with open(text_file, "w", encoding="utf-8") as f:
            f.write(result["text"])
        
        # Sauvegarde du SRT
        with open(srt_file, "w", encoding="utf-8") as f:
            for i, segment in enumerate(result["segments"]):
                start_time = format_timestamp(segment["start"])
                end_time = format_timestamp(segment["end"])
                text = segment["text"].strip()
                f.write(f"{i+1}\n{start_time} --> {end_time}\n{text}\n\n")
        
        print("✅ Sauvegarde terminée")
        return text_file, srt_file
        
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde : {e}")
        return None, None

print("✅ Fonctions de traitement définies")

## 5. Fonction de traitement unifié

In [ ]:
def process_file(file_path, temp_dir, whisper_model, device="auto"):
    """Traite un fichier unique (audio ou vidéo)"""
    print(f"\n🔧 Traitement de : {os.path.basename(file_path)}")
    
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    output_base = os.path.join(temp_dir, base_name)
    
    file_type = detect_file_type(file_path)
    print(f"📁 Type détecté : {file_type}")
    
    if file_type == "audio":
        # Fichier audio - transcription directe
        audio_source = file_path
    elif file_type == "video":
        # Fichier vidéo - conversion nécessaire
        audio_source = output_base + ".mp3"
        if not convert_video_to_audio(file_path, audio_source):
            return None, None
    else:
        print(f"❌ Type de fichier non supporté : {file_path}")
        return None, None
    
    # Transcription
    result = transcribe_audio(whisper_model, audio_source, device)
    if not result:
        return None, None
    
    # Sauvegarde
    text_path, srt_path = save_transcription(result, output_base)
    
    print(f"✅ Traitement terminé pour : {os.path.basename(file_path)}")
    return text_path, srt_path

def find_media_files(directory):
    """Trouve tous les fichiers média dans un répertoire"""
    extensions = ['*.mp4', '*.avi', '*.mkv', '*.webm', '*.mov', '*.flv',
                  '*.mp3', '*.wav', '*.aac', '*.flac', '*.ogg', '*.m4a']
    
    media_files = []
    for ext in extensions:
        media_files.extend(glob.glob(os.path.join(directory, '**', ext), recursive=True))
    
    return media_files

print("✅ Fonction de traitement unifié définie")

## 6. Interface utilisateur

In [ ]:
# Configuration du device
device = setup_device()

# Configuration des paramètres
temp_directory = '/kaggle/working'
whisper_model_name = "large-v3"

# Interface utilisateur simplifiée
print("🎯 WHISPER TRANSCRIPTION TOOL v2.0")
print("="*50)
print("Choisissez le type de source :")
print("1. 🎥 URL YouTube/vidéo")
print("2. 📁 Dossier de fichiers locaux")
print("3. 📄 Fichier unique (vidéo ou audio)")
print()

# Pour Kaggle, utiliser des variables pré-définies au lieu d'input()
# Modifiez ces variables selon vos besoins :

SOURCE_TYPE = "1"  # Changez cette valeur : "1", "2", ou "3"
SOURCE_INPUT = "https://www.youtube.com/watch?v=TLzna9__DnI"  # URL, chemin de dossier, ou chemin de fichier

print(f"Type sélectionné : {SOURCE_TYPE}")
print(f"Source : {SOURCE_INPUT}")
print()

## 7. Chargement du modèle Whisper

In [ ]:
print(f"🤖 Chargement du modèle Whisper '{whisper_model_name}' sur {device}...")

try:
    whisper_model = whisper.load_model(whisper_model_name, device=device)
    print("✅ Modèle Whisper chargé avec succès")
except Exception as e:
    print(f"❌ Erreur lors du chargement du modèle : {e}")
    raise

## 8. Traitement principal

In [ ]:
# Préparation du répertoire de travail
os.makedirs(temp_directory, exist_ok=True)
files_to_process = []
generated_files = []

print("🚀 Début du traitement...")
print("="*50)

# Gestion des différents types de sources
if SOURCE_TYPE == "1":  # URL
    print("📥 Mode : Téléchargement depuis URL")
    downloaded_file = download_video(SOURCE_INPUT, temp_directory)
    if downloaded_file:
        files_to_process.append(downloaded_file)
    else:
        print("❌ Échec du téléchargement")

elif SOURCE_TYPE == "2":  # Dossier
    print("📁 Mode : Traitement de dossier")
    if os.path.isdir(SOURCE_INPUT):
        files_to_process = find_media_files(SOURCE_INPUT)
        print(f"📊 {len(files_to_process)} fichiers trouvés")
    else:
        print(f"❌ Dossier introuvable : {SOURCE_INPUT}")

elif SOURCE_TYPE == "3":  # Fichier unique
    print("📄 Mode : Fichier unique")
    if os.path.isfile(SOURCE_INPUT):
        files_to_process.append(SOURCE_INPUT)
    else:
        print(f"❌ Fichier introuvable : {SOURCE_INPUT}")

else:
    print("❌ Type de source non valide")

print(f"\n📋 Nombre de fichiers à traiter : {len(files_to_process)}")
print()

## 9. Traitement des fichiers

In [ ]:
# Traitement de chaque fichier
for i, file_path in enumerate(files_to_process, 1):
    print(f"\n📁 Fichier {i}/{len(files_to_process)}")
    
    text_path, srt_path = process_file(file_path, temp_directory, whisper_model, device)
    
    if text_path:
        generated_files.append(text_path)
    if srt_path:
        generated_files.append(srt_path)

print("\n🎉 TRAITEMENT TERMINÉ !")
print("="*50)
print(f"📊 Fichiers traités : {len(files_to_process)}")
print(f"📝 Fichiers générés : {len(generated_files)}")
print()
print("📄 Fichiers de transcription générés :")
for file_path in generated_files:
    print(f"  ✅ {os.path.basename(file_path)}")

## 10. Création d'archive (optionnel)

In [ ]:
# Création d'une archive ZIP avec tous les fichiers générés
CREATE_ARCHIVE = True  # Changez à False si vous ne voulez pas d'archive

if CREATE_ARCHIVE and generated_files:
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    archive_path = os.path.join(temp_directory, f"transcriptions_{timestamp}.zip")
    
    print(f"\n📦 Création de l'archive : {os.path.basename(archive_path)}")
    
    try:
        with zipfile.ZipFile(archive_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file_path in generated_files:
                if os.path.exists(file_path):
                    zipf.write(file_path, os.path.basename(file_path))
        
        print(f"✅ Archive créée : {archive_path}")
        print(f"📊 Taille : {os.path.getsize(archive_path) / 1024:.1f} KB")
        
    except Exception as e:
        print(f"❌ Erreur lors de la création de l'archive : {e}")
else:
    print("\n📦 Création d'archive désactivée")

## 11. Nettoyage (optionnel)

In [ ]:
# Nettoyage des fichiers temporaires
CLEANUP_TEMP_FILES = False  # Changez à True pour nettoyer automatiquement

if CLEANUP_TEMP_FILES:
    print("\n🧹 Nettoyage des fichiers temporaires...")
    
    # Supprimer les fichiers audio/vidéo temporaires
    temp_extensions = ['.mp3', '.mp4', '.avi', '.mkv', '.webm']
    cleaned_count = 0
    
    for file in os.listdir(temp_directory):
        file_path = os.path.join(temp_directory, file)
        if any(file.endswith(ext) for ext in temp_extensions):
            try:
                os.remove(file_path)
                cleaned_count += 1
            except Exception as e:
                print(f"⚠️ Impossible de supprimer {file}: {e}")
    
    print(f"✅ {cleaned_count} fichiers temporaires supprimés")
else:
    print("\n🧹 Nettoyage automatique désactivé")
    print("Les fichiers sont conservés dans :", temp_directory)

## 12. Résumé final

In [ ]:
print("\n" + "="*60)
print("🎯 WHISPER TRANSCRIPTION TOOL v2.0 - RÉSUMÉ FINAL")
print("="*60)
print(f"⚡ Device utilisé : {device}")
print(f"🤖 Modèle Whisper : {whisper_model_name}")
print(f"📁 Type de source : {['', 'URL', 'Dossier', 'Fichier'][int(SOURCE_TYPE)]}")
print(f"📊 Fichiers traités : {len(files_to_process)}")
print(f"📝 Fichiers générés : {len(generated_files)}")
print(f"📂 Répertoire de sortie : {temp_directory}")
print("="*60)
print("✅ Traitement terminé avec succès !")

if generated_files:
    print("\n📋 Fichiers de transcription disponibles :")
    for file_path in generated_files:
        print(f"  📄 {os.path.basename(file_path)}")